O modelo tem o intuito de definir pontos criticos de acidentes com base na densidade, para isso é necessario definir o valor da densidade e o valor de numeros minimos

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

In [2]:
df_acidentes = pd.read_csv('Arquivos/df_acidentes_volume.csv', encoding='utf-8', sep=',')

Gerar uma coluna com numeros para gerar o loop no modelo

In [3]:
def coluna_em_numeros(df):
    concessionaria = {'Fernão Dias': 1,
                      'Litoral Sul': 2,
                      'Régis Bittencourt': 3,
                      'Planalto Sul': 4,
                      'Fluminense': 5}
    df['Concessionaria_1'] = df['Concessionaria'].replace(concessionaria).astype(int)

    rodovia = {'MG-BR381': 1,
               'MG-CONT': 3,
               'SP-BR381': 2,
               'BR101': 1,
               'BR116': 2,
               'BR376': 3,
               'CW-BR116': 1,
               'PR-BR116': 2,
               'SP-BR116': 3,
               'BR 101': 1,
               'BR116/PR': 1,
               'BR116/SC': 2}
    df['Rodovia_1'] = df['Rodovia'].replace(rodovia).astype(int)

    df['Sentido_1'] = df['Sentido'].replace('S', 1).replace('N', 2).astype(int)
    return df

df_acidentes = coluna_em_numeros(df_acidentes)

C:\Users\loany\AppData\Local\Temp\ipykernel_13044\2432938542.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Concessionaria_1'] = df['Concessionaria'].replace(concessionaria).astype(int)
C:\Users\loany\AppData\Local\Temp\ipykernel_13044\2432938542.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Rodovia_1'] = df['Rodovia'].replace(rodovia).astype(int)
C:\Users\loany\AppData\Local\Temp\ipykernel_13044\2432938542.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ver

In [4]:
df_acidentes.drop(columns='Veiculos').head()

,NumOcorrencia,DataOcorrencia,DescrOcorrencia,Sentido,Rodovia,Numveic,Ilesos,VitimasLeves,VitimasModeradas,VitimasGraves,...,Veiculo_Passageiro,Mes_Ano,Praca_pedagio,Volume_Equivalente,Volume_Total,Ano_Marginal,Marginal,Concessionaria_1,Rodovia_1,Sentido_1
0,15,2009-01-01,Acidente com VITIMA,S,BR101,1.0,NaN,1.0,NaN,NaN,...,False,2009-01-01,Praça 04,NaN,NaN,NaN,False,2,1,1
1,31,2009-01-01,Acidente com VITIMA,N,BR101,2.0,2.0,NaN,1.0,NaN,...,False,2009-01-01,Praça 03,NaN,NaN,NaN,False,2,1,2
2,231,2009-01-06,Acidente com VITIMA,S,BR101,1.0,NaN,1.0,NaN,NaN,...,False,2009-01-01,Praça 05,NaN,NaN,2009.0,True,2,1,1
3,150,2009-01-04,Acidente com Danos Materiais,S,BR376,1.0,1.0,NaN,NaN,NaN,...,False,2009-01-01,Praça 02,NaN,NaN,NaN,False,2,3,1
4,36,2009-01-07,Acidente com VITIMA,S,BR101,1.0,NaN,1.0,NaN,NaN,...,False,2009-01-01,Praça 04,NaN,NaN,NaN,False,2,1,1


Como o foco não são pedestres ou ciclistas, foram retirados do modelo

In [5]:
df_acidentes = df_acidentes.query('TipoAcidente != "Atropelamento - Pedestre" and TipoAcidente != "Atropelamento - Ciclista"')

Também não será avaliado acidentes antes de 2010, pois não ha informação do trafego

In [6]:
df_acidentes = df_acidentes.query('Ano >= 2010')

São tres loops, por concessionaria, por sentido, por rodovia

Foi escolhido o valor de 100 metros para densidade e 5 para numero minimo de pontos

In [7]:
df_acidentes['CssRodSentido'] = df_acidentes.apply(lambda x: '%s.%s.%s' % (x['Concessionaria'], x['Rodovia'], x['Sentido']), axis=1)

resumo_total = pd.DataFrame()
acidentes_total = pd.DataFrame()
ultimo_DBSCAN = 0

concessionaria = df_acidentes['Concessionaria_1'].unique()

for i in concessionaria:
    df0 = df_acidentes.query("Concessionaria_1 == @i").copy()
    sentido = df0['Sentido_1'].unique()
    for j in sentido:
        df1 = df0.query("Sentido_1 == @j").copy()
        rodovia = df1['Rodovia_1'].unique()
        for k in rodovia:
            df2 = df1.query("Rodovia_1 == @k").copy()
            X = df2[['Longitude', 'Latitude']]
            X = X.dropna()
            X = np.array(X)
            X = X.astype(float)

            modelo = DBSCAN(eps=(100/111320), min_samples=5).fit(X)

            class_predictions = modelo.labels_

            df2['CLUSTERS_DBSCAN'] = class_predictions
            y_pred = modelo.fit_predict(X)

            labels = modelo.labels_
            n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # quantidade de grupos
            n_noise_ = list(labels).count(-1)  # ruidos

            #armazenando informação de cada cluster, como rodovia, kmmt inicial e final, contagem de acidentes, etc.
            kmmin = df2.groupby(['Concessionaria', 'Sentido', 'Rodovia', 'CssRodSentido', 'CLUSTERS_DBSCAN'])['kmmt'].min().reset_index(name="kmmin")
            kmmin['CLUSTERS_DBSCAN'] = kmmin['CLUSTERS_DBSCAN'].apply(lambda x: x + ultimo_DBSCAN if x >= 0 else x)

            kmmax = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['kmmt'].max().reset_index(name="kmmax")
            kmmax.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)

            contagem = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['CLUSTERS_DBSCAN'].count().reset_index(name="contagem")
            contagem.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)


            somaUPS = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['UPS'].sum().reset_index(name="somaUPS")
            somaUPS.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)

            resumo = pd.concat([kmmin, kmmax, contagem, somaUPS], axis=1)
            resumo['extensao'] = resumo['kmmax'] - resumo['kmmin']
            
            #excluindo os ruidos
            resumo = resumo.query("CLUSTERS_DBSCAN >= 0")

            #devolvendo para o df_acidentes qual o numero do cluster que o acidente pertence, senão retorna vazio
            df2['CLUSTERS_DBSCAN'] = df2.apply(lambda row: resumo.loc[(resumo.CssRodSentido == row['CssRodSentido']) & (row['kmmt'] >= resumo.kmmin) &(row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].values[0] if not resumo.loc[(resumo.CssRodSentido == row['CssRodSentido']) & (row['kmmt'] >= resumo.kmmin) & (row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].empty else None, axis=1)

            df2.drop(["Sentido_1", "Rodovia_1", "Concessionaria_1"], axis=1, inplace=True)
            df2.reset_index(inplace=True)

            acidentes_total = pd.concat([acidentes_total, df2], axis=0, ignore_index=True)
            resumo_total = pd.concat([resumo_total, resumo], axis=0, ignore_index=True)
            
            #pego o ultimo valor do cluster nesse loop e utilizo como primeiro numero do proximo para não duplicar valor de cluster
            ultimo_DBSCAN = resumo_total['CLUSTERS_DBSCAN'].max() +1


In [8]:
acidentes_total['Cluster'] = np.where(acidentes_total['CLUSTERS_DBSCAN'].notna(), True, False)

In [9]:
acidentes_total.drop(columns='Veiculos').head()

,index,NumOcorrencia,DataOcorrencia,DescrOcorrencia,Sentido,Rodovia,Numveic,Ilesos,VitimasLeves,VitimasModeradas,...,Veiculo_Passageiro,Mes_Ano,Praca_pedagio,Volume_Equivalente,Volume_Total,Ano_Marginal,Marginal,CssRodSentido,CLUSTERS_DBSCAN,Cluster
0,416,24,2010-01-01,Acidente com Danos Materiais,N,BR101,1.0,1.0,NaN,NaN,...,False,2010-01-01,Praça 04,1112528.0,751003.0,NaN,False,Litoral Sul.BR101.N,NaN,False
1,418,539,2010-01-02,Acidente com VITIMA,N,BR101,1.0,NaN,NaN,2.0,...,False,2010-01-01,Praça 03,1038955.0,626942.0,NaN,False,Litoral Sul.BR101.N,1.0,True
2,419,1,2010-01-03,Acidente com VITIMA,N,BR101,1.0,NaN,1.0,NaN,...,False,2010-01-01,Praça 05,869020.5,569853.0,2009.0,True,Litoral Sul.BR101.N,NaN,False
3,420,147,2010-01-03,Acidente com VITIMA,N,BR101,1.0,NaN,1.0,1.0,...,False,2010-01-01,Praça 04,1112528.0,751003.0,2009.0,True,Litoral Sul.BR101.N,3.0,True
4,421,203,2010-01-03,Acidente com VITIMA,N,BR101,1.0,NaN,2.0,NaN,...,False,2010-01-01,Praça 03,1038955.0,626942.0,NaN,False,Litoral Sul.BR101.N,NaN,False


In [10]:
resumo_total

,Concessionaria,Sentido,Rodovia,CssRodSentido,CLUSTERS_DBSCAN,kmmin,kmmax,contagem,somaUPS,extensao
0,Litoral Sul,N,BR101,Litoral Sul.BR101.N,0,128.0,128.0,67,26.0,0.0
1,Litoral Sul,N,BR101,Litoral Sul.BR101.N,1,54.7,54.8,23,15.0,0.1
2,Litoral Sul,N,BR101,Litoral Sul.BR101.N,2,204.1,204.2,78,75.0,0.1
3,Litoral Sul,N,BR101,Litoral Sul.BR101.N,3,136.8,137.0,157,96.0,0.2
4,Litoral Sul,N,BR101,Litoral Sul.BR101.N,4,57.7,57.7,17,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...
8509,Fernão Dias,S,SP-BR381,Fernão Dias.SP-BR381.S,8509,49.6,49.6,5,20.0,0.0
8510,Fernão Dias,S,SP-BR381,Fernão Dias.SP-BR381.S,8510,48.6,48.7,7,13.0,0.1
8511,Fernão Dias,S,SP-BR381,Fernão Dias.SP-BR381.S,8511,82.3,82.3,7,13.0,0.0
8512,Fernão Dias,S,SP-BR381,Fernão Dias.SP-BR381.S,8512,23.1,23.1,8,17.0,0.0


In [11]:
acidentes_total.to_csv('Arquivos/acidentes_cluster.csv')
resumo_total.to_csv('Arquivos/resumo_cluster.csv')